In [1]:
import os
import cv2
import torch
import numpy as np
import torch.nn as nn

from tqdm import tqdm
from torch.nn import functional as F 
 
class ResBlock(nn.Module):
    def __init__(self, num_channel):
        super(ResBlock, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(num_channel, num_channel, 3, 1, 1),
            nn.BatchNorm2d(num_channel),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_channel, num_channel, 3, 1, 1),
            nn.BatchNorm2d(num_channel))
        self.activation = nn.ReLU(inplace=True)

    def forward(self, inputs):
        output = self.conv_layer(inputs)
        output = self.activation(output + inputs)
        return output


class DownBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(DownBlock, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, 3, 2, 1),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channel, out_channel, 3, 1, 1),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True))


    def forward(self, inputs):
        output = self.conv_layer(inputs)
        return output


class UpBlock(nn.Module):
    def __init__(self, in_channel, out_channel, is_last=False):
        super(UpBlock, self).__init__()
        self.is_last = is_last
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channel, in_channel, 3, 1, 1),
            nn.BatchNorm2d(in_channel),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(in_channel, out_channel, 3, 1, 1))
        self.act = nn.Sequential(
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True))
        self.last_act = nn.Tanh()


    def forward(self, inputs):
        output = self.conv_layer(inputs)
        if self.is_last:
            output = self.last_act(output)
        else:
            output = self.act(output)
        return output



class SimpleGenerator(nn.Module):
    def __init__(self, num_channel=32, num_blocks=4):
        super(SimpleGenerator, self).__init__()
        self.down1 = DownBlock(3, num_channel)
        self.down2 = DownBlock(num_channel, num_channel*2)
        self.down3 = DownBlock(num_channel*2, num_channel*3)
        self.down4 = DownBlock(num_channel*3, num_channel*4)
        res_blocks = [ResBlock(num_channel*4)]*num_blocks
        self.res_blocks = nn.Sequential(*res_blocks)
        self.up1 = UpBlock(num_channel*4, num_channel*3)
        self.up2 = UpBlock(num_channel*3, num_channel*2)
        self.up3 = UpBlock(num_channel*2, num_channel)
        self.up4 = UpBlock(num_channel, 3, is_last=True)

    def forward(self, inputs):
        down1 = self.down1(inputs)
        down2 = self.down2(down1)
        down3 = self.down3(down2)
        down4 = self.down4(down3)
        down4 = self.res_blocks(down4)
        up1 = self.up1(down4)
        up2 = self.up2(up1+down3)
        up3 = self.up3(up2+down2)
        up4 = self.up4(up3+down1)
        return up4


In [ ]:
import tkinter 
import cv2
import os
import PIL.Image, PIL.ImageTk
import time

class App:
    def __init__(self, window, window_title, video_source=0):
        self.window = window
        self.window.title(window_title)
        self.video_source = video_source

        # open video source (by default this will try to open the computer webcam)
        self.vid = MyVideoCapture(self.video_source)

        # Create a canvas that can fit the above video source size
        self.canvas = tkinter.Canvas(window, width = window.winfo_screenwidth(), height = window.winfo_screenheight())
        self.canvas.pack(fill='both', expand=True)
        bg = PIL.ImageTk.PhotoImage(PIL.Image.open("ss.png"))
        self.canvas.create_image(0, 0, image=bg, anchor='nw')
 
        ts=time.strftime("%d-%m-%Y-%H-%M-%S")
        # Button that lets the user take a snapshot
        self.btn_snapshot=tkinter.Button(window, text="Click a Snapshot", width=50, command=lambda: self.snapshot(ts))
        button1_window = self.canvas.create_window(800, 600, anchor='center', window=self.btn_snapshot)
#         self.btn_snapshot.pack(anchor=tkinter.CENTER, expand=True)
        # After it is called once, the update method will be automatically called every delay milliseconds
        self.delay = 15
        self.update()
 
        self.window.mainloop()
    
    def emoj(self,ts):
        weight = torch.load('weight.pth', map_location='cpu')
        model = SimpleGenerator()
        model.load_state_dict(weight)
        #torch.save(model.state_dict(), 'weight.pth')
        model.eval()
        raw_image = cv2.imread("images/frame-" +  ts + ".jpg")
#         cv2.imshow("cropped", raw_image)
        raw_image = cv2.resize(raw_image, (256,256), interpolation = cv2.INTER_AREA)
        image = raw_image/127.5 - 1 
        image = image.transpose(2, 0, 1) 
        image = torch.tensor(image).unsqueeze(0)
        output = model(image.float())
        output = output.squeeze(0).detach().numpy()
        output = output.transpose(1, 2, 0)
        output = (output + 1) * 127.5
        output = np.clip(output, 0, 255).astype(np.uint8)
        output = np.concatenate([raw_image, output], axis=0)
        cv2.imwrite("results/frame-" +  ts + ".jpg", output)
        cv2.imshow("emojy", output)
        
    def faceid(self,ts):
        import cv2
        face_cascade = cv2.CascadeClassifier(r"haarcascade_frontalface_default.xml")
        img = cv2.imread("input/frame-" +  ts + ".jpg")
#         print(img)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.05, minNeighbors=5)
 
        for x, y, w, h in faces:
            img = cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
        resized = cv2.resize(img, (int(img.shape[1]), int(img.shape[0])))
        cv2.imshow("Gray", resized)
        crop_img = img[y:y+h, x:x+w]
#         cv2.imshow("cropped", crop_img)
        cv2.imwrite("images/frame-" + ts + ".jpg", crop_img)
        self.emoj(ts)
        cv2.waitKey(0)
        cv2.destroyAllWindows() 
 
    def snapshot(self,ts):
        # Get a frame from the video source
        ret, frame = self.vid.get_frame()
        path = 'input'
        if ret:
            cv2.imwrite("input/frame-" + ts + ".jpg", cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        self.faceid(ts)
#         self.emoj(ts)
 

    def update(self):
        # Get a frame from the video source
        ret, frame = self.vid.get_frame()

        if ret:
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
            self.canvas.create_image(0, 0, image = self.photo, anchor = tkinter.NW)

        self.window.after(self.delay, self.update)
        
class MyVideoCapture:
    def __init__(self, video_source=0):
        # Open the video source
        self.vid = cv2.VideoCapture(video_source)
        if not self.vid.isOpened():
            raise ValueError("Unable to open video source", video_source)

        # Get video source width and height
        self.width = self.vid.get(cv2.CAP_PROP_FRAME_WIDTH)
        self.height = self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT)

    def get_frame(self):
        if self.vid.isOpened():
            ret, frame = self.vid.read()
            if ret:
                # Return a boolean success flag and the current frame converted to BGR
                return (ret, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            else:
                return (ret, None)
        else:
            return (ret, None)

    # Release the video source when the object is destroyed
    def __del__(self):
        if self.vid.isOpened():
            self.vid.release()


# Create a window and pass it to the Application object
App(tkinter.Toplevel(), "Emojify")